In [33]:
from sage.algebras.flag_algebras import *

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
TG.exclude([k4,k4m])

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoK4m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))

11 106 8157


In [34]:
from itertools import combinations
def get_supergraphs(base):
    missing_edges = []
    for e in combinations(range(6), 3):
        f = [v for v in e]
        if f not in base:
            missing_edges.append(f)
            
    graphs = []
    for i in range(len(missing_edges)):
        for E in combinations(missing_edges, i):
            M = base + [e for e in E]
            k4_ = False
            for t in combinations(range(6), 4):
                numb_edges = 0
                for e in M:
                    counts = True
                    for v in e:
                        if v not in t:
                            counts = False
                            break
                    if counts:
                        numb_edges += 1
                if numb_edges >= 3: # Contains K_4^-
                    k4_ = True
                    break
            if not k4_:
                graphs.append(M)
    return graphs

In [39]:
base_graph = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [1, 2, 4], [1, 2, 5]]

f411 = TGp(6, ftype = [0, 1, 2], edges = base_graph)

for graph in get_supergraphs(base_graph):
    f411 = f411 + TGp(6, ftype=[0, 1, 2], edges=graph)
    
pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])

p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]

assump = [pointed_edge-2/7, degree_difference, -degree_difference]

min_f411 = TGp.optimize_problem(f411, 6, maximize=False, positives=assump)

print("\n\n", min_f411)

Ftypes constructed in 1.63s
Block sizes done in 0.00s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 156.94it/s]
Table calculation done in 0.03s
Target and constraint calculation done in 0.04s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.59e-01 Pobj: -6.3827467e+01 Ad: 3.06e-01 Dobj:  4.6571185e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -9.4435189e+01 Ad: 7.13e-01 Dobj:  2.1695925e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0103848e+02 Ad: 9.07e-01 Dobj:  8.3219921e-03 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0751904e+02 Ad: 6.99e-01 Dobj:  7.2481846e-03 
Iter:  5 Ap: 5.68e-01 Pobj: -7.6098426e+01 Ad: 4.88e-01 Dobj:  8.4818514e-03 
Iter:  6 Ap: 5.94e-01 Pobj: -6.2467210e+01 Ad: 6.82e-01 Dobj:  9.8153225e-03 
Iter:  7 Ap: 9.82e-01 Pobj: -4.2833687e+01 Ad: 6.81e-01 Dobj:  7.7351145e-03 
Iter:  8 Ap: 9.12e-01 Pobj: -2.4949

In [36]:
base_graph = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [1, 2, 4], [0, 2, 5]]

f321 = TGp(6, ftype = [0, 1, 2], edges = base_graph)

for graph in get_supergraphs(base_graph):
    f321 = f321 + TGp(6, ftype=[0, 1, 2], edges=graph)

max_f321 = TGp.optimize_problem(f321, 6, maximize=True, positives=assump)

print("\n\n", max_f321)

Ftypes constructed in 0.17s
Block sizes done in 0.00s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 152.59it/s]
Table calculation done in 0.03s
Target and constraint calculation done in 0.04s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.60e-01 Pobj: -6.3702202e+01 Ad: 3.07e-01 Dobj:  1.3132608e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -9.3988520e+01 Ad: 7.15e-01 Dobj:  5.7762478e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0052436e+02 Ad: 9.07e-01 Dobj: -4.3423618e-03 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0689583e+02 Ad: 6.97e-01 Dobj: -9.3506608e-03 
Iter:  5 Ap: 5.61e-01 Pobj: -7.5681878e+01 Ad: 4.85e-01 Dobj: -1.3938466e-02 
Iter:  6 Ap: 5.96e-01 Pobj: -6.2108017e+01 Ad: 6.83e-01 Dobj: -2.5177181e-02 
Iter:  7 Ap: 9.83e-01 Pobj: -4.2680866e+01 Ad: 6.80e-01 Dobj: -4.9118361e-02 
Iter:  8 Ap: 9.16e-01 Pobj: -2.4881

In [37]:
base_graph = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [0, 2, 4], [0, 1, 5]]

f222 = TGp(6, ftype = [0, 1, 2], edges = base_graph)

for graph in get_supergraphs(base_graph):
    f222 = f222 + TGp(6, ftype=[0, 1, 2], edges=graph)

max_f222 = TGp.optimize_problem(f222, 6, maximize=True, positives=assump)

print("\n\n", max_f222)

Ftypes constructed in 0.15s
Block sizes done in 0.00s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 152.28it/s]
Table calculation done in 0.03s
Target and constraint calculation done in 1.43s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 0.11118888666752288
Iter:  1 Ap: 8.38e-01 Pobj: -6.7180055e+01 Ad: 2.81e-01 Dobj: -3.0412419e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.0437176e+02 Ad: 6.87e-01 Dobj:  1.6061968e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.1245191e+02 Ad: 9.07e-01 Dobj: -3.8115110e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -1.2120664e+02 Ad: 7.17e-01 Dobj: -5.5373406e-02 
Iter:  5 Ap: 6.73e-01 Pobj: -9.0352980e+01 Ad: 5.46e-01 Dobj: -7.2640617e-02 
Iter:  6 Ap: 5.60e-01 Pobj: -7.3021069e+01 Ad: 6.40e-01 Dobj: -9.5591496e-02 
Iter:  7 Ap: 9.49e-01 Pobj: -4.9916571e+01 Ad: 6.83e-01 Dobj: -4.1823221e-02 
Iter:

In [38]:
base_graph = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [0, 2, 4], [0, 1, 5]]

f222 = TGp(6, ftype = [0, 1, 2], edges = base_graph)

for graph in get_supergraphs(base_graph):
    f222 = f222 + TGp(6, ftype=[0, 1, 2], edges=graph)

max_f222 = TGp.optimize_problem((f321 + f222)/2, 6, maximize=True, positives=assump)

print("\n\n", max_f222)

Ftypes constructed in 0.15s
Block sizes done in 0.00s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 179.39it/s]
Table calculation done in 0.02s
Target and constraint calculation done in 0.04s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.53e-01 Pobj: -6.4610693e+01 Ad: 3.00e-01 Dobj: -3.2917156e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -9.6765740e+01 Ad: 7.07e-01 Dobj:  9.8379428e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0372217e+02 Ad: 9.07e-01 Dobj: -2.1577283e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -1.1078037e+02 Ad: 7.03e-01 Dobj: -3.2396007e-02 
Iter:  5 Ap: 6.02e-01 Pobj: -7.8806263e+01 Ad: 5.07e-01 Dobj: -4.3764094e-02 
Iter:  6 Ap: 5.83e-01 Pobj: -6.4644847e+01 Ad: 6.74e-01 Dobj: -5.8670685e-02 
Iter:  7 Ap: 9.79e-01 Pobj: -4.3857343e+01 Ad: 6.84e-01 Dobj: -4.5382867e-02 
Iter:  8 Ap: 9.21e-01 Pobj: -2.5261